WHAT IS DARLI

WHAT TO EXPECT NEXT

The model in DARLi is a building block that can be further extended with external forces and contacts, different selector matrices, friction in joints, state space and parametric representations, please follow up to the next steps of tutorial to get in to that.

- General-info
- Backends
- Models 
- Floating Base,Conventions 
- Further Read and Structure

- Model of Poly-Articulated system
- Conventions
- rnea and aba
- Floating Base



OTHER TUTORS:
* MODELS, BODIES, SELECTORS ETC
* FUNCTIONAL, CODE GEN, BATCHED
* STATE SPACE 
* PARAMETRIC REPRESENTATIONS


## **Introduction to DARLi, Basics of Backends and Models** 


The DARLi (Differentiable Articulated Robotics Library) is a wrapper around the CasADi kinematics/dynamics and Pinocchio libraries. Its primary goal is to facilitate the creation of differentiable models for robotic systems, provided a URDF (Unified Robot Description Format) file. **DARLi is not an implementation of mechanics-oriented** algorithms, such as Featherstone's Articulated Body or the Recursive Newton-Euler. Instead, we rely on the efficient implementations provided by [Pinocchio](https://github.com/stack-of-tasks/pinocchio/tree/master) and offer a wrapper that grants easy access to a suite of features that we and our colleagues have found useful in practice of trajectory planning, feedback control, and system identification over poly-articulated robots.

We will use [robot_descriptions.py](https://github.com/robot-descriptions/robot_descriptions.py) library to simplify the process of urdf retrieval.

**Note**: You have to restart kernel after `robot_descriptions` is installed

In [1]:

%%capture
!pip3 install robot_descriptions
from robot_descriptions import panda_description, panda_mj_description

### Backends 
The typical DARLi model is based on instance of Backend. Backend take `urdf` file as input and defines the way of computing two main functions: Articulated Bpdy Algorithm `aba` and Recursive Newton Euler `rnea`, as well as severals utility functions. These are soledly based on Pinnochio. For now there are two backends are available:

TODO: WHAT IS RNEA AND ABA

- `PinocchioBackend` - Provide interface to the Pinnochio python bindings. And may be useufull if one wants to obtain fast **numericall** computations. 
- `CasadiBackend` - The CasADi wrapper around Pinnochiio, this is used to cimpute dynamics in **symbolical** fashion, this allows for seeamless calculation of derivative and potentially used in trajectory optimization and optimal control.


The acces to backends is as simple as:

In [2]:
from darli.backend import CasadiBackend, PinocchioBackend

# Building the Pinnochio backend
pin_backend = PinocchioBackend(panda_description.URDF_PATH)
# Building the CasADi backend
cas_backend = CasadiBackend(panda_mj_description.MJCF_PATH)

In [3]:
cas_backend.nq

9

In [4]:
print(panda_description.URDF_PATH)

/home/simeon/.cache/robot_descriptions/example-robot-data/robots/panda_description/urdf/panda.urdf


In [5]:
print(panda_mj_description.MJCF_PATH)

/home/simeon/.cache/robot_descriptions/mujoco_menagerie/franka_emika_panda/panda.xml


In [6]:
cas_backend.nq

9

In [7]:
pin_backend.nq
pin_backend.nv
pin_backend.aba
pin_backend.rnea

<bound method PinocchioBackend.rnea of <darli.backend._pinocchio.PinocchioBackend object at 0x7ac82c2b4370>>

### Floating Base, Fixed joints and Conventions: 



TODO: Something on floating base and conventions

In [8]:
from robot_descriptions import go2_description

In [9]:
go2_cas = CasadiBackend(go2_description.URDF_PATH)

In [10]:
print(f"Dimensions of model, nq: {go2_cas.nq} nv: {go2_cas.nv}")

Dimensions of model, nq: 12 nv: 12


In [11]:
from darli.backend import JointType

go2_cas_floating = CasadiBackend(
    go2_description.URDF_PATH, root_joint=JointType.FREE_FLYER
)

print(
    f"Dimensions of floating model, nq: {go2_cas_floating.nq} nv: {go2_cas_floating.nv}"
)

Dimensions of floating model, nq: 19 nv: 18


In [12]:
cas_backend = CasadiBackend(go2_description.URDF_PATH, root_joint=JointType.FREE_FLYER)

The model in DARLi is a building block that can be further extended with external forces and contacts, different selector matrices, friction in joints, state space and parametric representations, please follow up to the next steps of tutorial to get in to that.

- General-info
- Backends
- Models 
- Floating Base,Conventions 
- Further Read and Structure


- Model of Poly-Articulated system
- Conventions
- rnea and aba
- Floating Base


### Conventions



### Backends

In [13]:
from darli.model import Model

In [14]:
casadi_model = Model(cas_backend)

In [15]:
casadi_model.com()

CoM(position=SX(@1=1, @2=2, @3=(@2*q_4), @4=(@3*q_4), @5=(@2*q_5), @6=(@5*q_5), @7=0.0036612, @8=cos(q_17), @9=-0.00430848, @10=0.00084392, @11=cos(q_18), @12=-0.02623, @13=sin(q_18), @14=(@9+((@10*@11)+(@12*@13))), @15=sin(q_17), @16=-0.0376704, @17=-0.041322, @18=(@16+(((@12*@11)-(@10*@13))+@17)), @19=-0.391442, @20=cos(q_14), @21=cos(q_15), @22=sin(q_15), @23=(@9+((@10*@21)+(@12*@22))), @24=sin(q_14), @25=(@16+(((@12*@21)-(@10*@22))+@17)), @26=-0.0036612, @27=cos(q_11), @28=cos(q_12), @29=sin(q_12), @30=(@9+((@10*@28)+(@12*@29))), @31=sin(q_11), @32=(@16+(((@12*@28)-(@10*@29))+@17)), @33=0.391442, @34=cos(q_8), @35=cos(q_9), @36=sin(q_9), @37=(@9+((@10*@35)+(@12*@36))), @38=sin(q_8), @39=(@16+(((@12*@35)-(@10*@36))+@17)), @40=((((0.146694+((@7+((@8*@14)+(@15*@18)))+@19))+((@7+((@20*@23)+(@24*@25)))+@19))+((@26+((@27*@30)+(@31*@32)))+@33))+((@26+((@34*@37)+(@38*@39)))+@33)), @41=(@3*q_3), @42=(@5*q_6), @43=-0.104019, @44=cos(q_16), @45=sin(q_16), @46=-7.119e-05, @47=(@46+((@8*@18)-(@

One can retrive basic model info as follows:

In [16]:
nq = casadi_model.nq  # dimensionality of configuration
nv = casadi_model.nv  # dimensionality of generilized velocities
nu = casadi_model.nu  # dimensionality  of control inputs
q_min, q_max = (
    casadi_model.q_min,
    casadi_model.q_max,
)  # minimal and maximal limits on joint pos
nq, nv, nu

(19, 18, 18)

In [17]:
joint_names = casadi_model.joint_names  # names of the joints
joint_names

['universe',
 'root_joint',
 'FL_hip_joint',
 'FL_thigh_joint',
 'FL_calf_joint',
 'FR_hip_joint',
 'FR_thigh_joint',
 'FR_calf_joint',
 'RL_hip_joint',
 'RL_thigh_joint',
 'RL_calf_joint',
 'RR_hip_joint',
 'RR_thigh_joint',
 'RR_calf_joint']

### **Equations of Motion and Dynamics**

The dynamics of articulated mechanics in robotic systems is usually represented as:
$$
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{C}(\mathbf{q},\mathbf{v})\mathbf{v} + \mathbf{g}(\mathbf{q})  = 
\mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{c}(\mathbf{q},\mathbf{v}) + \mathbf{g}(\mathbf{q}) = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \mathbf{Q}
$$

where:
* $\mathbf{Q} \in \mathbb{R}^{nv}$ - generalized forces corresponding to generilized coordinates
* $\mathbf{q} \in \mathbb{R}^{nq}$ - vector of generilized coordinates
* $\mathbf{v} \in \mathbb{R}^{nq}$ - vector of generilized velocities (sometimes $\mathbf{v} = \dot{\mathbf{q}}$, but not in case of $\mathbf{q}$ containing quaternions)
* $\mathbf{M} \in \mathbb{R}^{nv \times nv}$ - positive definite symmetric inertia matrix 
* $\mathbf{c} \in \mathbb{R}^{nv}$ - describe centrifugal and Coriolis forces
* $\mathbf{C} \in \mathbb{R}^{nv \times nv}$ - describe 'coefficients' of centrifugal and Coriolis forces
* $\mathbf{g} \in \mathbb{R}^{nv}$ - describes effect of gravity and other position depending forces
* $\mathbf{h} \in \mathbb{R}^{nv} $ - combined effect of $\mathbf{g}$ and $\mathbf{c}$


One can get all of the above quantities in symbotics as follows:

In [18]:
inertia = casadi_model.inertia
gravity_vector = casadi_model.gravity
bias_force = casadi_model.bias_force
coriolis_matrix = casadi_model.coriolis_matrix
coriolis = casadi_model.coriolis

Each of the above define the CasAdi functions:

And can be evaluated both numerically and Functionalally:

In [19]:
inertia = casadi_model.inertia
import numpy as np
import casadi as cs

# Functionalal computation
print("Functional:", inertia(cs.SX.sym("q", nq)))
# Numerical computation
# print("Numerical:", inertia(np.random.randn(nq)))

Functional: @1=15.019, @2=0, @3=0.865237, @4=0.844248, @5=0.815514, @6=0.226221, @7=-0.0513926, @8=sin(q_16), @9=cos(q_16), @10=-3.51729e-05, @11=0.66502, @12=cos(q_17), @13=-0.0279869, @14=0.144131, @15=-0.213, @16=-0.135206, @17=cos(q_18), @18=0.0043501, @19=sin(q_18), @20=(@15+((@16*@17)-(@18*@19))), @21=(@13+(@14*@20)), @22=sin(q_17), @23=-0.00320095, @24=((@18*@17)+(@16*@19)), @25=(@23+(@14*@24)), @26=((@12*@21)-(@22*@25)), @27=(@10+(@11*@26)), @28=((@7*@8)+(@9*@27)), @29=(-0.00415649+(@6*@28)), @30=0.184486, @31=0.0513926, @32=sin(q_13), @33=cos(q_13), @34=cos(q_14), @35=cos(q_15), @36=sin(q_15), @37=(@15+((@16*@35)-(@18*@36))), @38=(@13+(@14*@37)), @39=sin(q_14), @40=((@18*@35)+(@16*@36)), @41=(@23+(@14*@40)), @42=((@34*@38)-(@39*@41)), @43=(@10+(@11*@42)), @44=((@31*@32)+(@33*@43)), @45=((@5*@29)+(@30*@44)), @46=0.155752, @47=sin(q_10), @48=cos(q_10), @49=cos(q_11), @50=cos(q_12), @51=sin(q_12), @52=(@15+((@16*@50)-(@18*@51))), @53=(@13+(@14*@52)), @54=sin(q_11), @55=((@18*@50)

Note that above are calculated not via Lagrange formalism but by using efficient recursive Newton-Euler algorithm (for `bias_force`), while inertia matrix is evaluated by composite rigid body algorithm (CRBA)

There are some notions apart from dynamical coefficients that may be useful as well, such as com kinematics and jacobians, and energy:

In [20]:
casadi_model.energy()

Energy(kinetic=SX(@1=2, @2=-0.00537167, @3=0.0211894, @4=((@2*v_3)-(@3*v_5)), @5=(@3*v_4), @6=(@2*v_4), @7=-6.923, @8=0.678, @9=0.0465, @10=(v_0-(@9*v_5)), @11=cos(q_7), @12=0.1934, @13=(v_1+(@12*v_5)), @14=sin(q_7), @15=(v_2-((@12*v_4)-(@9*v_3))), @16=((@11*@13)+(@14*@15)), @17=((@11*@15)-(@14*@13)), @18=0.00194, @19=((@11*v_5)-(@14*v_4)), @20=-0.000105, @21=((@11*v_4)+(@14*v_5)), @22=((@18*@19)-(@20*@21)), @23=(v_6+v_3), @24=-0.0054, @25=((@20*@23)-(@24*@19)), @26=((@24*@21)-(@18*@23)), @27=-0.678, @28=0.00048, @29=0.000884, @30=0.000596, @31=-3.01e-06, @32=1.11e-06, @33=-1.42e-06, @34=1.152, @35=cos(q_8), @36=0.0955, @37=(@10-(@36*@19)), @38=sin(q_8), @39=(@17+(@36*@23)), @40=((@35*@37)-(@38*@39)), @41=((@38*@37)+(@35*@39)), @42=-0.0223, @43=((@38*@23)+(@35*@19)), @44=-0.0327, @45=(v_7+@21), @46=((@42*@43)-(@44*@45)), @47=((@35*@23)-(@38*@19)), @48=-0.00374, @49=((@44*@47)-(@48*@43)), @50=((@48*@45)-(@42*@47)), @51=-1.152, @52=0.00584, @53=0.0058, @54=0.00103, @55=8.72e-05, @56=-0.0

In [21]:
com_position = casadi_model.com().position
com_velocity = casadi_model.com().velocity
com_jacobian = casadi_model.com().jacobian
com_jacobian_dt = casadi_model.com().jacobian_dt
potential_energy = casadi_model.energy().potential
kinetic_energy = casadi_model.energy().kinetic
lagrangian = casadi_model.lagrangian()

# TODO
# momentum

In [22]:
com_jacobian_dt

SX(@1=6.923, @2=0.0665823, @3=2, @4=(@3*q_4), @5=(@4*q_3), @6=(@3*q_5), @7=(@6*q_6), @8=(@2*(@5-@7)), @9=(@6*q_3), @10=(@4*q_6), @11=(@2*(@9+@10)), @12=(@1*((v_5*@8)-(v_4*@11))), @13=0.678, @14=cos(q_7), @15=sin(q_7), @16=((@14*v_5)-(@15*v_4)), @17=((@15*@11)+(@14*@8)), @18=((@14*v_4)+(@15*v_5)), @19=((@14*@11)-(@15*@8)), @20=(@13*((@16*@17)-(@18*@19))), @21=cos(q_8), @22=1.152, @23=sin(q_8), @24=(v_6+v_3), @25=((@23*@24)+(@21*@16)), @26=(v_7+@18), @27=1, @28=(@4*q_4), @29=(@6*q_5), @30=(@2*(@27-(@28+@29))), @31=((@21*@19)+(@23*@30)), @32=(@22*((@25*@17)-(@26*@31))), @33=cos(q_9), @34=0.194, @35=sin(q_9), @36=((@21*@24)-(@23*@16)), @37=((@35*@36)+(@33*@25)), @38=(v_8+@26), @39=((@21*@30)-(@23*@19)), @40=((@33*@31)+(@35*@39)), @41=(@34*((@37*@17)-(@38*@40))), @42=(@34*@40), @43=(@41+(v_8*@42)), @44=((@33*@39)-(@35*@31)), @45=((@33*@36)-(@35*@25)), @46=(@34*((@38*@44)-(@45*@17))), @47=(@34*@44), @48=(@46-(v_8*@47)), @49=((@33*@43)+(@35*@48)), @50=(@22*@31), @51=((@50+(@33*@42))-(@35*@47)

#### **Forward and Inverse Dynamics**


In inverse dynamics we looking for the generilized forces:
$$
    \mathbf{Q} = \mathbf{M}(\mathbf{q})\dot{\mathbf{v}} + \mathbf{h}(\mathbf{q},\mathbf{v}) = \text{rnea}(\mathbf{q}, \mathbf{v}, \dot{\mathbf{v}})
$$

In [23]:
casadi_model.contact_qforce

SX(@1=0, [@1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1])

In [24]:
casadi_model.inverse_dynamics()

SX(@1=6.923, @2=9.81, @3=2, @4=(@3*q_5), @5=(@3*q_4), @6=(((((v_1*v_5)-(v_2*v_4))+((v_4*v_2)-(v_5*v_1)))+dv_0)+(@2*((@4*q_3)-(@5*q_6)))), @7=-0.00537167, @8=(@1*(@6+(@7*dv_4))), @9=0.0211894, @10=(@1*(v_2-(@9*v_4))), @11=(@1*(v_1-((@7*v_3)-(@9*v_5)))), @12=0.678, @13=-0.0465, @14=(@6-(@13*dv_5)), @15=-0.00194, @16=cos(q_16), @17=sin(q_16), @18=((@16*v_4)+(@17*v_5)), @19=(((@16*dv_5)-(@17*dv_4))-(v_15*@18)), @20=-0.000105, @21=((@16*v_5)-(@17*v_4)), @22=((v_15*@21)+((@16*dv_4)+(@17*dv_5))), @23=(@12*(@14-((@15*@19)-(@20*@22)))), @24=-0.1934, @25=(v_2-((@24*v_4)-(@13*v_3))), @26=(v_1+(@24*v_5)), @27=((@16*@25)-(@17*@26)), @28=0.0054, @29=(v_15+v_3), @30=(@12*(@27-((@28*@18)-(@15*@29)))), @31=((@16*@26)+(@17*@25)), @32=(@12*(@31-((@20*@29)-(@28*@21)))), @33=cos(q_17), @34=1.152, @35=-0.0955, @36=(@14-(@35*@19)), @37=sin(q_17), @38=(@3*q_3), @39=(((((v_0*v_4)-(v_1*v_3))+((v_3*v_1)-(v_4*v_0)))+dv_2)+(@2*(1-((@38*q_3)+(@5*q_4))))), @40=(@39-((@24*dv_4)-(@13*dv_3))), @41=(((((v_2*v_3)-(v_0*v_

While forward dynamics is basically solving the equations of motion with respect to accelerations $\dot{\mathbf{v}}$. However the solution of the above in general case usually yields the complexity of $O(nv^3)$, for this reason a way to go is to use celebrated Featherstone Articulated Body algorithm (ABA), which effectevely exploit sparsity of $\mathbf{M}$ for the tree structures and produce nearly linear complexity $O(nv)$:


$$
\dot{\mathbf{v}} = \mathbf{M}^{-1}(\mathbf{q})(\mathbf{Q} - \mathbf{h}(\mathbf{q},\mathbf{v})) = \text{aba}(\mathbf{q}, \mathbf{v}, \mathbf{Q})
$$

<!-- Articulated body algorithm 

Feather stone algorithm
 -->
<!-- http://gamma.cs.unc.edu/courses/robotics-f08/LEC/ArticulatedBodyDynamics.pdf -->

In [25]:
casadi_model.contact_qforce

SX(@1=0, [@1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1, @1])

In [26]:
casadi_model.forward_dynamics()

SX(@1=6.923, @2=0.678, @3=cos(q_17), @4=1.152, @5=0.0554451, @6=cos(q_18), @7=-0.00445784, @8=sin(q_18), @9=((@5*@6)+(@7*@8)), @10=0.193857, @11=((@7*@6)+(@10*@8)), @12=((@9*@6)+(@11*@8)), @13=-0.0376704, @14=-0.213, @15=(@14*@12), @16=(@13+@15), @17=0.00704794, @18=(@17+(@14*@15)), @19=(@16/@18), @20=((@4+@12)-(@19*@16)), @21=sin(q_17), @22=((@7*@6)-(@5*@8)), @23=((@10*@6)-(@7*@8)), @24=0.00430848, @25=((@11*@6)-(@9*@8)), @26=(@24+(@14*@25)), @27=(@26/@18), @28=(((@22*@6)+(@23*@8))-(@27*@16)), @29=((@3*@20)+(@21*@28)), @30=(@25-(@19*@26)), @31=((@4+((@23*@6)-(@22*@8)))-(@27*@26)), @32=((@3*@30)+(@21*@31)), @33=((@29*@3)+(@32*@21)), @34=-0.0955, @35=((@3*@28)-(@21*@20)), @36=((@3*@31)-(@21*@30)), @37=((@35*@3)+(@36*@21)), @38=-6.14238e-06, @39=0.000149952, @40=((@38*@6)+(@39*@8)), @41=-0.000102677, @42=1.5274e-06, @43=((@41*@6)+(@42*@8)), @44=((@40*@6)+(@43*@8)), @45=8.8791e-06, @46=(@45+(@14*@44)), @47=(@44-(@19*@46)), @48=0.0256896, @49=((@39*@6)-(@38*@8)), @50=((@42*@6)-(@41*@8)), @

#### **Passive Joints and Selector**

In practical situations 

Choosing the passive joints:

In [27]:
import numpy as np

print(f"Old input dimensions: {casadi_model.nu}")
S = np.random.randn(casadi_model.nv, casadi_model.nv + 2)
casadi_model.update_selector(S)
print(f"New input dimensions: {casadi_model.nu}")

Old input dimensions: 18
New input dimensions: 20


In [28]:
casadi_model.qfrc_u

SX([tau_0, tau_1, tau_2, tau_3, tau_4, tau_5, tau_6, tau_7, tau_8, tau_9, tau_10, tau_11, tau_12, tau_13, tau_14, tau_15, tau_16, tau_17, tau_18, tau_19])

In [29]:
casadi_model.forward_dynamics

<bound method Model.forward_dynamics of <darli.model._model.Model object at 0x7ac82424c3d0>>

In [30]:
casadi_model.update_selector(passive_joints=range(3))
print(f"New input dimensions: {casadi_model.nu}")
print(f"New selector:\n {casadi_model.selector}")

New input dimensions: 15
New selector:
 [[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [31]:
casadi_model.forward_dynamics

<bound method Model.forward_dynamics of <darli.model._model.Model object at 0x7ac82424c3d0>>

#### **Bodies and Contacts**

In [32]:
print(casadi_model.backend.description_path)

/home/simeon/.cache/robot_descriptions/unitree_ros/robots/go2_description/urdf/go2_description.urdf


In [33]:
casadi_model.add_body(["FL_calf"])
casadi_model.bodies

{'FL_calf': <darli.model._body.Body at 0x7ac82424dcc0>}

In [34]:
model.body("link06")

NameError: name 'model' is not defined

### Frames

In [ ]:
model.body("link06").linear_acceleration

In [ ]:
model.body("link06").linear_acceleration.local

In [ ]:
model.body("link06").jacobian.local

In [ ]:
model.body("link03").jacobian.local
model.body("link03").jacobian_dt.local
model.body("link03").linear_velocity.local
model.body("link03").angular_velocity.local
model.body("link03").linear_acceleration.local
model.body("link03").angular_acceleration.local

The body jacobian and velocities can be calculated with respect to `world`, `local` and `world_aligned` frames.

In [ ]:
model.body("link06").jacobian.world

Note that body name can be initialized with dictionary that maps given name to one presented in urdf i.e: `{'ee':'link06'}`

##### **Contacts**

In [ ]:
model.body("link06").add_contact("wrench")

In [ ]:
model.body("link06").contact.dim
# model.body("link06").contact.contact_frame
model.body("link06").contact.ref_frame
model.body("link06").contact.qforce

Do not forget to rebuild the model:

In [ ]:
# model.update_model()

Note how arguments are changed in dynamics related functions, i.e:

In [ ]:
model.forward_dynamics

the state space representation and jacobians are changed as well:

In [ ]:
model.state_space.state_derivative

In [ ]:
model.state_space.state_jacobian

In [ ]:
model.body("link06").contact.add_cone(mu=0.5, X=0.05, Y=0.02)

In [ ]:
wrench_cone = model.body("link06").contact.cone.full()

wrench_cone

In [ ]:
model.body("link06").contact.cone.linear()

In [ ]:
model.add_body({"ee": "link06"})
model.bodies

One can add bodies on the initialization stage based on following syntax:

In [ ]:
# Symbolic(z1_description.URDF_PATH, bodies_names={'shoulder':'link03', 'ee':'link06'})

The `bodies_names` arguments can be listof body names present in urdf, however for increased readability we suggest to use the dictionary as shown above.